In [1]:
# encoding: utf-8 
"""
@author: 周世聪
@contact: abner1zhou@gmail.com 
@file: data_processing.py 
@time: 2019/12/4 下午9:23 
@desc: 数据处理函数，包括分词，词向量构建，embedding_matrix，训练集 测试集创建
"""

import pandas as pd
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
import jieba
import re

import sys
sys.path.append("..")
from utils import multi_cpus
from utils import config


def clean_sentence(sentence):
    """
    特殊符号去除
    使用正则表达式去除无用的符号、词语
    """
    if isinstance(sentence, str):
        return re.sub(
            r'[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】“”！，。？、~@#￥%……&*（）]+|车主说|技师说|语音|图片|你好|您好|nan',
            '', sentence)
    else:
        return ''


def get_stop_words(stop_words_path):
    """
    处理停用词表
    :param stop_words_path: 停用词表路径
    :return: 停用词表list
    """
    stop = []
    with open(stop_words_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            stop.append(line.strip())
    return stop


# cut函数，分别对question，dialogue，report进行切词
def cut_words(sentences):
    # 清除无用词
    sentence = clean_sentence(sentences)
    # 切词，默认精确模式，全模式cut参数cut_all=True
    words = jieba.cut(sentence)
    # 过滤停用词
    stop_words = get_stop_words(config.stop_word_path)
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)


def cut_data_frame(df):
    """
    数据集批量处理方法
    :param df: 数据集
    :return:处理好的数据集
    """
    # 批量预处理 训练集和测试集
    for col_name in ['Brand', 'Model', 'Question', 'Dialogue']:
        df[col_name] = df[col_name].apply(cut_words)

    if 'Report' in df.columns:
        # 训练集 Report 预处理
        df['Report'] = df['Report'].apply(cut_words)
    return df


def get_segment(data_path):
    """
    将输入的数据集切词并返回
    :param data_path: 输入csv格式数据集
    :return: 返回一个切词完毕的数据集
    """
    df = pd.read_csv(data_path)
    df = df.dropna()
    # 1.切词
    seg_df = multi_cpus.parallelize(df, cut_data_frame)
    # 对切词完的数据进行拼接,这部分用来训练词向量
    seg_df["Data_X"] = seg_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
    if "Report" in seg_df.columns:
        seg_df['merged'] = seg_df[['Question', 'Dialogue', 'Report']].apply(lambda x: ' '.join(x), axis=1)  # axis 横向拼接
        seg_df['Data_Y'] = seg_df[['Report']]
    else:
        seg_df['merged'] = seg_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)  # axis 横向拼接

    return seg_df






1. 对原始数据集进行切词、删除停用词、无用字符等操作

In [ ]:
train_seg_df = get_segment(config.train_data_path)
test_seg_df = get_segment(config.test_data_path)

2. 合并数据集

In [3]:
merged_df = pd.concat([train_seg_df[['merged']], test_seg_df[['merged']]], axis=0)

In [4]:
merged_df.to_csv(config.merger_seg_path, header=False, index=False)

In [5]:
def get_w2v(merger_seg_path):
    w2v_model = word2vec.Word2Vec(LineSentence(merger_seg_path), 
                            workers=6, 
                            min_count=5, # 忽略词频小于5的单词
                            size=200)
    return w2v_model

3. 计算词向量

In [28]:
w2v_model = get_w2v(config.merger_seg_path)

4. 获得训练集、测试集数据

In [10]:
import numpy as np

In [11]:
def get_max_len(data):
    """
    获得合适的最大长度值
    :param data: 待统计的数据  train_df['Question']
    :return: 最大长度值
    """
    # TODO FIX len size bug
    max_lens = data.apply(lambda x: x.count(' ') + 1)
    return int(np.mean(max_lens) + 2 * np.std(max_lens))

In [12]:
def pad_proc(sentence, max_len, vocab):
    '''
    # 填充字段
    < start > < end > < pad > < unk > max_lens
    '''
    # 0.按空格统计切分出词
    words = sentence.strip().split(' ')
    # 1. 截取规定长度的词数
    words = words[:max_len]
    # 2. 填充< unk > ,判断是否在vocab中, 不在填充 < unk >
    sentence = [word if word in vocab else '<UNK>' for word in words]
    # 3. 填充< start > < end >
    sentence = ['<START>'] + sentence + ['<STOP>']
    # 4. 判断长度，填充　< pad >
    sentence = sentence + ['<PAD>'] * (max_len - len(words))
    return ' '.join(sentence)

In [7]:
train_seg_df['X'] = train_seg_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
test_seg_df['X'] = test_seg_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
train_seg_df['Y'] = train_seg_df[['Report']]

In [32]:
train_x_max_len = get_max_len(train_seg_df['X'])
train_y_max_len = get_max_len(train_seg_df['Y'])
test_x_max_len = get_max_len(test_seg_df['X'])

5. 对训练集进行转换

因为训练词向量的时候忽略了词频小于5的单词，所以需要额外的填充字符

并且encoder的单元数是固定的，每段输入数据的长度需要设定为统一。多的忽略，少的填充字段

In [33]:
train_seg_df['X'] = train_seg_df['X'].apply(lambda x: pad_proc(x, train_x_max_len, vocab))
train_seg_df['Y'] = train_seg_df['Y'].apply(lambda x: pad_proc(x, train_x_max_len, vocab))
test_seg_df['X'] = test_seg_df['X'].apply(lambda x: pad_proc(x, train_x_max_len, vocab))

In [34]:
train_seg_df['X'].to_csv(config.train_x_path, header=False, index=False)
train_seg_df['Y'].to_csv(config.train_y_path, header=False, index=False)
test_seg_df['X'].to_csv(config.test_x_path, header=False, index=False)

6. 再次训练词向量，把刚才的填充字段加上去

In [35]:
wv_train_epochs = 10

In [36]:
# update word vector
print('start retrain w2v model')
w2v_model.build_vocab(LineSentence(config.train_x_path), update=True)
w2v_model.train(LineSentence(config.train_x_path), epochs=wv_train_epochs, total_examples=w2v_model.corpus_count)
print('1/3')
w2v_model.build_vocab(LineSentence(config.train_y_path), update=True)
w2v_model.train(LineSentence(config.train_y_path), epochs=wv_train_epochs, total_examples=w2v_model.corpus_count)
print('2/3')
w2v_model.build_vocab(LineSentence(config.test_x_path), update=True)
w2v_model.train(LineSentence(config.test_x_path), epochs=wv_train_epochs, total_examples=w2v_model.corpus_count)

start retrain w2v model
1/3
2/3


(17254508, 50966850)

In [38]:
w2v_model.save(config.w2v_model_path)

7. 构建词表

In [39]:
vocab = {word : index for index, word in enumerate(w2v_model.wv.index2word)}
reverse_vocab = {index : word for index, word in enumerate(w2v_model.wv.index2word)}

最终train_X 需要转换成index， 把数据集里面的单词用index来表示出来，这样可以在encoder的时候index 得到word vector

In [41]:
def translate_data(sentence, vocab):
    words = sentence.split(' ')
    ids = [vocab[word] if word in vocab else vocab['<UNK>'] for word in words]
    return ids

In [42]:
train_x_ids = train_seg_df['X'].apply(lambda x: translate_data(x, vocab))

In [46]:
train_x_ids

0        [33060, 33061, 33061, 398, 985, 244, 229, 398,...
1        [33060, 33061, 33061, 770, 33061, 382, 215, 35...
2        [33060, 33061, 33061, 1480, 97, 461, 33061, 68...
3        [33060, 33061, 33061, 14428, 9, 303, 61, 526, ...
4        [33060, 33061, 33061, 1391, 97, 770, 12661, 33...
                               ...                        
82938    [33060, 33061, 33061, 457, 208, 326, 198, 326,...
82939    [33060, 33061, 33061, 1251, 97, 1190, 5231, 65...
82940    [33060, 33061, 33061, 211, 976, 50, 160, 334, ...
82941    [33060, 33061, 33061, 13128, 3123, 4064, 25277...
82942    [33060, 33061, 33061, 2820, 68, 1041, 1690, 18...
Name: X, Length: 81572, dtype: object

In [47]:
"wv_train_epochs".upper

<function str.upper()>

In [51]:
a = "wv_train_epochs".upper()

In [52]:
a

'WV_TRAIN_EPOCHS'

In [50]:
print(a)

<built-in method upper of str object at 0x7ff65f6cc430>
